In [ ]:
import os
import re
import sys
import glob
import json
import shutil
import skimage
import imageio
import tifffile
import datetime
import requests
import jsonschema
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors

from skimage import feature
from skimage import morphology

%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('/Users/keith.cheveralls/projects/opencell/')
from opencell.imaging import images, utils, viz

sys.path.append('/Users/keith.cheveralls/projects/dragonfly-automation/')
import dragonfly_automation.utils
from dragonfly_automation.fov_models import PipelineFOVScorer
from dragonfly_automation.qc import half_plate_layout
from dragonfly_automation.qc.pipeline_plate_qc import PipelinePlateQC

In [ ]:
# resize images for the SOP
jpgs = glob.glob('/Users/keith.cheveralls/Box/KCC-slides/automated-pipeline-microscopy-SOP/*.JPG')
for jpg in jpgs:
    pass
    im = imageio.imread(jpg)
    imageio.imsave(jpg.replace('.JPG', '_RESZ4.JPG'), skimage.transform.downscale_local_mean(im, (4, 4, 1)))

In [ ]:
# local acquisition directories (logs only)
pml_dirs = glob.glob('/Users/keith.cheveralls/image-data/raw-pipeline-microscopy/PML*/')
pml_dirs = sorted(pml_dirs)
pml_dirs

In [ ]:
# real acquisition directories (on ESS)
pml_dirs = glob.glob('/Volumes/ml_group/raw-pipeline-microscopy/PML*')
pml_dirs = sorted(pml_dirs)
pml_dirs

### Create the external_metadata files for existing experiments

In [ ]:
# load the google sheet export
google_sheet = pd.read_csv('/Users/keith.cheveralls/Downloads/Pipeline-microscopy-master-key - PMLs (1).csv')
google_sheet.rename(columns={'id': 'pml_id'}, inplace=True)

In [ ]:
# generate the metadata JSON files from the google sheet
# (note that these files are created when the acquisition is initialized; 
# they were regenerated to eliminate variation in e.g., parental_line_id)

# the metadata attributes for canonical half-plate or full-plate imaging
columns = ['pml_id', 'parental_line', 'plate_id', 'platemap_type', 'imaging_round_id']

overwrite = False
for path in pml_dirs:
    
    # the directory name is exactly the pml_id
    pml_id = path.split(os.sep)[-1]
    
    # retrieve the entry in the google sheet for this pml_id
    if pml_id not in google_sheet.pml_id.values:
        print('pml_id %s not found in google sheet' % pml_id)
        continue
    metadata = google_sheet.loc[google_sheet.pml_id==pml_id].iloc[0][columns]
   
    # if there is a custom platemap, the metadata should contain only the pml_id and the platemap_type
    if metadata['platemap_type'] == 'custom':
        metadata = metadata[['pml_id', 'platemap_type']]

    dst_filepath = os.path.join(path, 'metadata.json')
    if os.path.isfile(dst_filepath) and not overwrite:
        print('metadata.json already exists in %s' % path)
    else:
        with open(dst_filepath, 'w') as file:
            json.dump(dict(metadata), file)

### Load all of the existing external metadata files

In [ ]:
rows = []
for path in pml_dirs:
    dirname = path.split(os.sep)[-1]
    filepath = glob.glob(os.path.join(path, 'metadata.json'))
    if len(filepath) != 1:
        print('More than one metadata file in %s' % path)

    with open(filepath.pop()) as file:
        metadata = json.load(file)
        if metadata.get('pml_id') is not None and metadata['pml_id'] != dirname:
            print('Warning')
            
        row = {'dirname': dirname}
        row.update(metadata)
        rows.append(row)

In [ ]:
all_md = pd.DataFrame(data=rows)
all_md

In [ ]:
for pml_dir in pml_dirs:
    qc = PipelinePlateQC(pml_dir)
    qc.summarize()
    print('\n')

In [ ]:
qc = PipelinePlateQC('/Volumes/ml_group/raw-pipeline-microscopy/PML0233/')
qcc = PipelinePlateQC('/Volumes/ml_group/raw-pipeline-microscopy/PML0231/')

### File renaming

In [ ]:
# raw filename format
'czML0383-P0001-A01-PML0300-S16__MMStack_16-B9-16.ome.tif'
'czML0383-P0001-A06-PML0300-S05__MMStack_1180-G9-5.ome.tif'

# oc-plate-microscopy format
'czML0383-P0001-A01-PML0300-S01_ATL2.tif'

In [ ]:
qcc.summarize()

In [ ]:
qcc.plot_counts_and_scores()

In [ ]:
qcc.platemap

In [ ]:
qc.aq_log.head()

In [ ]:
# get target names from API
result = requests.get('http://localhost:5000/lines?plate_id=P0019')
pd.DataFrame(data=result.json())[['cell_line_id', 'plate_id', 'well_id', 'target_name']].sort_values(by='well_id')

In [ ]:
qc.rename_raw_tiffs(preview=True)

In [ ]:
qc.generate_z_projections('/Users/keith.cheveralls/projects/opencell-process/')